In [1]:
from models.baseline.base.decision_tree import decision_tree_100hz, decision_tree_50hz, decision_tree_25hz, decision_tree_20hz, decision_tree_10hz
from models.baseline.base.random_forest import random_forest_100hz, random_forest_50hz, random_forest_25hz, random_forest_20hz, random_forest_10hz

model_setups = [
    (decision_tree_100hz, 100, (0,0), 'decision_tree'),
    (decision_tree_50hz, 50, (0,1), 'decision_tree'),
    (decision_tree_25hz, 25, (0,2), 'decision_tree'),
    (decision_tree_20hz, 20, (0,3), 'decision_tree'),
    (decision_tree_10hz, 10, (0,4), 'decision_tree'),
    (random_forest_100hz, 100, (1,0), 'random_forest'),
    (random_forest_50hz, 50, (1,1), 'random_forest'),
    (random_forest_25hz, 25, (1,2), 'random_forest'),
    (random_forest_20hz, 20, (1,3), 'random_forest'),
    (random_forest_10hz, 10, (1,4), 'random_forest')
]

In [ ]:
validation_results_circle = []

fig, ax = plt.subplots(ncols=2, nrows=5, sharey=True, sharex=True, figsize=(30,25))
fig.tight_layout()

for setup in model_setups:
    MODEL = setup[0]
    FREQ = setup[1]
    STEP = (1000 / FREQ)
    COL = setup[2][0]
    ROW = setup[2][1]

    df_val = pd.read_csv('data/validation/move_circle_20210522_1.csv').set_index('ms')

    # initialize empty dataset to collect results
    inf_results = pd.DataFrame([],columns=['start','end','result'])

    # treat dataset 
    df_downsampled = downsample_df(df_val, STEP) # downsample dataset
    df_inference = transform_data_for_inference(df_downsampled) # converts dataset to inference format

    # Plot signals
    ax[ROW][COL].plot(df_downsampled)

    # generate a list of steps    
    inference_step = list(np.arange(0, df_val.index[-1] + 1 - 1010, step=STEP))

    for st in inference_step:
        res = np.argmax(run_inference(df_inference, MODEL, st, STEP))
        inf_results = pd.concat([inf_results, pd.DataFrame([{'start':st,'end':st+1000,'result':res}])], axis=0)

        if res in [1,2,3]:
            color = line_color(res)
            ax[ROW][COL].axvline(x=st+500, ymin=0, ymax=0.4, color=color, alpha=0.4)

    # Create Legend
    blue_patch = mpatches.Patch(color='blue', label='X Movement')
    red_patch = mpatches.Patch(color='red', label='Y Movement')
    green_patch = mpatches.Patch(color='green', label='Circle Movement')
    fig.legend(handles=[blue_patch, red_patch, green_patch])


    # get error rate
    wrong_percentage = calculate_error(inf_results, 'circle')

    validation_results_circle.append({'model':setup[-1], 'hz':setup[1], 'wrong_percentage': wrong_percentage, 'value_counts': inf_results['result'].value_counts()})